<a href="https://colab.research.google.com/github/oaarnikoivu/dissertation/blob/master/Emotional_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting Emotions from Tweets with BERT 

First we need to install the transformers python package to get access to the pre-trained BERT models.

In [46]:
!pip install transformers
!pip install fast-bert

In [0]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
from box import Box
from tqdm import tqdm, trange
from pathlib import Path
from sklearn.model_selection import train_test_split

import torch
import pandas as pd 
import collections 
import random
import numpy as np

import logging
import datetime

from torch.nn import BCEWithLogitsLoss
from fast_bert.modeling import BertForMultiLabelSequenceClassification
from fast_bert.data_cls import BertDataBunch, InputExample, InputFeatures, MultiLabelTextProcessor, convert_examples_to_features
from fast_bert.learner_cls import BertLearner
from fast_bert.metrics import accuracy_multilabel, accuracy_thresh, fbeta, roc_auc

In [48]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
torch.cuda.empty_cache()

In [0]:
run_start_time = datetime.datetime.today().strftime('%Y-%m-%d_%H-%M-%S')

In [0]:
path = '/content/drive/My Drive/'

DATA_PATH = Path(path + '/datasets/SemEval/')
LABEL_PATH = Path(path + '/labels/')

MODEL_PATH = Path(path + '/models/')
LOG_PATH = Path(path + '/logs/')

MODEL_PATH.mkdir(exist_ok=True)
LOG_PATH.mkdir(exist_ok=True)

BERT_PRETRAINED_PATH = Path(path + '/BERT/')

OUTPUT_PATH = Path(MODEL_PATH/'output')
OUTPUT_PATH.mkdir(exist_ok=True)

# BERT Arguments

In [0]:
args = Box({
    "multi_gpu": False,
    "multi_label": True,
    "batch_size_per_gpu": 16,
    "warmup_steps": 500,
    "max_seq_length": 128,
    "num_epochs": 6, 
    "is_fp16": False,
    "logging_steps": 50,
    "schedule_type": 'warmup_cosine',
    "optimizer_type": 'lamb',
    "learning_rate": 6e-5
})

# GPU

In [0]:
device = torch.device('cuda')

# check if multiple GPUs are available

if torch.cuda.device_count() > 1:
    args.multi_gpu = True
else:
    args.multi_gpu = False

# Custom BERT Models

In [0]:
class MyMultiLabelBertClassifier(BertForSequenceClassification):
  """Bert model for classification."""

  def forward(
      self,
      input_ids,
      token_type_ids=None,
      attention_mask=None,
      labels=None,
      position_ids=None,
      head_mask=None,
  ):

    outputs = self.bert(
        input_ids, 
        position_ids=position_ids,
        token_type_ids=token_type_ids,
        attention_mask=attention_mask,
        head_mask=head_mask
    )
    pooled_output = outputs[1]

    pooled_output = self.dropout(pooled_output)
    logits = self.classifier(pooled_output)

    # add hidden states and attention if they are here
    outputs = (logits,) + outputs[2:]

    if labels is not None:
      loss_fct = BCEWithLogitsLoss()

      loss = loss_fct(
          logits.view(-1, self.num_labels), labels.view(-1, self.num_labels)
      )
      outputs = (loss,) + outputs
    
    return outputs 

In [0]:
from fast_bert.data_cls import MODEL_CLASSES as MCd 

In [73]:
MCd['bert']

(transformers.configuration_bert.BertConfig,
 transformers.modeling_bert.BertForSequenceClassification,
 transformers.tokenization_bert.BertTokenizer)

In [0]:
MCd['custom_bert'] = (BertConfig,
                                BertForSequenceClassification,
                                BertTokenizer)

In [75]:
MCd

{'albert': (transformers.configuration_albert.AlbertConfig,
  transformers.modeling_albert.AlbertForSequenceClassification,
  transformers.tokenization_albert.AlbertTokenizer),
 'bert': (transformers.configuration_bert.BertConfig,
  transformers.modeling_bert.BertForSequenceClassification,
  transformers.tokenization_bert.BertTokenizer),
 'camembert': (transformers.configuration_camembert.CamembertConfig,
  transformers.modeling_camembert.CamembertForSequenceClassification,
  transformers.tokenization_camembert.CamembertTokenizer),
 'custom_bert': (transformers.configuration_bert.BertConfig,
  transformers.modeling_bert.BertForSequenceClassification,
  transformers.tokenization_bert.BertTokenizer),
 'distilbert': (transformers.configuration_distilbert.DistilBertConfig,
  transformers.modeling_distilbert.DistilBertForSequenceClassification,
  transformers.tokenization_distilbert.DistilBertTokenizer),
 'roberta': (transformers.configuration_roberta.RobertaConfig,
  transformers.modeling_

In [0]:
from fast_bert.learner_cls import MODEL_CLASSES as MCl

In [77]:
MCl

{'albert': (transformers.configuration_albert.AlbertConfig,
  (transformers.modeling_albert.AlbertForSequenceClassification,
   fast_bert.modeling.AlbertForMultiLabelSequenceClassification),
  transformers.tokenization_albert.AlbertTokenizer),
 'bert': (transformers.configuration_bert.BertConfig,
  (transformers.modeling_bert.BertForSequenceClassification,
   fast_bert.modeling.BertForMultiLabelSequenceClassification),
  transformers.tokenization_bert.BertTokenizer),
 'camembert': (transformers.configuration_camembert.CamembertConfig,
  (transformers.modeling_camembert.CamembertForSequenceClassification,
   fast_bert.modeling.CamembertForMultiLabelSequenceClassification),
  transformers.tokenization_camembert.CamembertTokenizer),
 'custom_bert': (transformers.configuration_bert.BertConfig,
  (transformers.modeling_bert.BertForSequenceClassification,
   __main__.MyMultiLabelBertClassifier),
  transformers.tokenization_bert.BertTokenizer),
 'distilbert': (transformers.configuration_disti

In [0]:
MCl['custom_bert'] = (BertConfig,
                      (BertForSequenceClassification, MyMultiLabelBertClassifier),
                      BertTokenizer)

In [79]:
MCl['custom_bert']

(transformers.configuration_bert.BertConfig,
 (transformers.modeling_bert.BertForSequenceClassification,
  __main__.MyMultiLabelBertClassifier),
 transformers.tokenization_bert.BertTokenizer)

In [0]:
label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 
              'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# FastBert DataBunch Object

In [81]:
databunch = BertDataBunch(DATA_PATH, 
                          LABEL_PATH,
                          tokenizer='bert-base-uncased',
                          train_file='train.csv',
                          val_file='val.csv',
                          test_data='test.csv',
                          label_file='labels.csv',
                          text_col="Tweet",
                          label_col=label_cols,
                          batch_size_per_gpu=args.batch_size_per_gpu,
                          max_seq_length=args.max_seq_length,
                          multi_gpu=args.multi_gpu,
                          multi_label=True,
                          model_type='custom_bert')

01/17/2020 15:01:51 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/17/2020 15:01:51 - INFO - root -   Loading features from cached file /content/drive/My Drive/datasets/SemEval/cache/cached_custom_bert_train_multi_label_128_train.csv
01/17/2020 15:01:51 - INFO - root -   Loading features from cached file /content/drive/My Drive/datasets/SemEval/cache/cached_custom_bert_dev_multi_label_128_val.csv
01/17/2020 15:01:51 - INFO - root -   Loading features from cached file /content/drive/My Drive/datasets/SemEval/cache/cached_custom_bert_test_multi_label_128_test


In [82]:
databunch.train_dl.dataset[0][3]

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1.])

In [83]:
num_labels = len(databunch.labels)
num_labels

11

# Evaluation Metrics

In [0]:
metrics = []
metrics.append({'name': 'accuracy_thresh', 'function': accuracy_thresh})
metrics.append({'name': 'roc_auc', 'function': roc_auc})
metrics.append({'name': 'fbeta', 'function': fbeta})

# Define FastBert Learner

In [85]:
learner = BertLearner.from_pretrained_model(
    databunch,
    pretrained_path='bert-base-uncased',
    metrics=metrics,
    device=device,
    logger=logger,
    output_dir=OUTPUT_PATH,
    finetuned_wgts_path=None,
    warmup_steps=args.warmup_steps,
    multi_gpu=args.multi_gpu,
    is_fp16=args.is_fp16,
    multi_label=args.multi_label,
    logging_steps=args.logging_steps)

01/17/2020 15:01:54 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
01/17/2020 15:01:54 - INFO - transformers.configuration_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 11,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pru

bert-base-uncased
<class 'str'>


01/17/2020 15:01:57 - INFO - transformers.modeling_utils -   Weights of MyMultiLabelBertClassifier not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
01/17/2020 15:01:57 - INFO - transformers.modeling_utils -   Weights from pretrained model not used in MyMultiLabelBertClassifier: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']


# Train the model

In [86]:
learner.fit(args.num_epochs, 
            args.learning_rate, 
            schedule_type=args.schedule_type,
            optimizer_type=args.optimizer_type,
            validate=True)

01/17/2020 15:02:00 - INFO - __main__ -   ***** Running training *****
01/17/2020 15:02:00 - INFO - __main__ -     Num examples = 6838
01/17/2020 15:02:00 - INFO - __main__ -     Num Epochs = 6
01/17/2020 15:02:00 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 16
01/17/2020 15:02:00 - INFO - __main__ -     Gradient Accumulation steps = 1
01/17/2020 15:02:00 - INFO - __main__ -     Total optimization steps = 2568


01/17/2020 15:02:15 - INFO - __main__ -   Running evaluation
01/17/2020 15:02:15 - INFO - __main__ -     Num examples = 886
01/17/2020 15:02:15 - INFO - __main__ -     Batch size = 32


01/17/2020 15:02:19 - INFO - __main__ -   eval_loss after step 50: 0.7069349735975266: 
01/17/2020 15:02:19 - INFO - __main__ -   eval_accuracy_thresh after step 50: 0.4897393584251404: 
01/17/2020 15:02:19 - INFO - __main__ -   eval_roc_auc after step 50: 0.5051216858329646: 
01/17/2020 15:02:19 - INFO - __main__ -   eval_fbeta after step 50: 0.5426353216171265: 
01/17/2020 15:02:19 - INFO - __main__ -   lr after step 50: 6e-06
01/17/2020 15:02:19 - INFO - __main__ -   train_loss after step 50: 0.7110221040248871
01/17/2020 15:02:33 - INFO - __main__ -   Running evaluation
01/17/2020 15:02:33 - INFO - __main__ -     Num examples = 886
01/17/2020 15:02:33 - INFO - __main__ -     Batch size = 32


01/17/2020 15:02:37 - INFO - __main__ -   eval_loss after step 100: 0.6992753595113754: 
01/17/2020 15:02:37 - INFO - __main__ -   eval_accuracy_thresh after step 100: 0.507387638092041: 
01/17/2020 15:02:37 - INFO - __main__ -   eval_roc_auc after step 100: 0.504594573241654: 
01/17/2020 15:02:37 - INFO - __main__ -   eval_fbeta after step 100: 0.5335439443588257: 
01/17/2020 15:02:37 - INFO - __main__ -   lr after step 100: 1.2e-05
01/17/2020 15:02:37 - INFO - __main__ -   train_loss after step 100: 0.7046544277667999
01/17/2020 15:02:51 - INFO - __main__ -   Running evaluation
01/17/2020 15:02:51 - INFO - __main__ -     Num examples = 886
01/17/2020 15:02:51 - INFO - __main__ -     Batch size = 32


01/17/2020 15:02:55 - INFO - __main__ -   eval_loss after step 150: 0.6876016216618674: 
01/17/2020 15:02:55 - INFO - __main__ -   eval_accuracy_thresh after step 150: 0.5534577965736389: 
01/17/2020 15:02:55 - INFO - __main__ -   eval_roc_auc after step 150: 0.5029914249640889: 
01/17/2020 15:02:55 - INFO - __main__ -   eval_fbeta after step 150: 0.5220840573310852: 
01/17/2020 15:02:55 - INFO - __main__ -   lr after step 150: 1.8e-05
01/17/2020 15:02:55 - INFO - __main__ -   train_loss after step 150: 0.693045687675476
01/17/2020 15:03:10 - INFO - __main__ -   Running evaluation
01/17/2020 15:03:10 - INFO - __main__ -     Num examples = 886
01/17/2020 15:03:10 - INFO - __main__ -     Batch size = 32


...

01/17/2020 15:17:08 - INFO - __main__ -   eval_loss after step 2450: 0.37185040967805044: 
01/17/2020 15:17:08 - INFO - __main__ -   eval_accuracy_thresh after step 2450: 0.8496819138526917: 
01/17/2020 15:17:08 - INFO - __main__ -   eval_roc_auc after step 2450: 0.8586223631787452: 
01/17/2020 15:17:08 - INFO - __main__ -   eval_fbeta after step 2450: 0.6493012309074402: 
01/17/2020 15:17:08 - INFO - __main__ -   lr after step 2450: 4.80718323058289e-07
01/17/2020 15:17:08 - INFO - __main__ -   train_loss after step 2450: 0.37581997334957123
01/17/2020 15:17:22 - INFO - __main__ -   Running evaluation
01/17/2020 15:17:22 - INFO - __main__ -     Num examples = 886
01/17/2020 15:17:22 - INFO - __main__ -     Batch size = 32


01/17/2020 15:17:26 - INFO - __main__ -   eval_loss after step 2500: 0.371839131627764: 
01/17/2020 15:17:26 - INFO - __main__ -   eval_accuracy_thresh after step 2500: 0.8497844934463501: 
01/17/2020 15:17:26 - INFO - __main__ -   eval_roc_auc after step 2500: 0.8586181841032239: 
01/17/2020 15:17:26 - INFO - __main__ -   eval_fbeta after step 2500: 0.6495723724365234: 
01/17/2020 15:17:26 - INFO - __main__ -   lr after step 2500: 1.5992690042286096e-07
01/17/2020 15:17:26 - INFO - __main__ -   train_loss after step 2500: 0.3710510492324829
01/17/2020 15:17:41 - INFO - __main__ -   Running evaluation
01/17/2020 15:17:41 - INFO - __main__ -     Num examples = 886
01/17/2020 15:17:41 - INFO - __main__ -     Batch size = 32


01/17/2020 15:17:44 - INFO - __main__ -   eval_loss after step 2550: 0.3718365100877626: 
01/17/2020 15:17:44 - INFO - __main__ -   eval_accuracy_thresh after step 2550: 0.8497844934463501: 
01/17/2020 15:17:44 - INFO - __main__ -   eval_roc_auc after step 2550: 0.8586241324223965: 
01/17/2020 15:17:44 - INFO - __main__ -   eval_fbeta after step 2550: 0.6495575904846191: 
01/17/2020 15:17:44 - INFO - __main__ -   lr after step 2550: 1.1215222253061485e-08
01/17/2020 15:17:44 - INFO - __main__ -   train_loss after step 2550: 0.37104988932609556
01/17/2020 15:17:49 - INFO - __main__ -   Running evaluation
01/17/2020 15:17:49 - INFO - __main__ -     Num examples = 886
01/17/2020 15:17:49 - INFO - __main__ -     Batch size = 32


01/17/2020 15:17:53 - INFO - __main__ -   eval_loss after epoch 6: 0.3718364941222327: 
01/17/2020 15:17:53 - INFO - __main__ -   eval_accuracy_thresh after epoch 6: 0.8497844934463501: 
01/17/2020 15:17:53 - INFO - __main__ -   eval_roc_auc after epoch 6: 0.8586241934307983: 
01/17/2020 15:17:53 - INFO - __main__ -   eval_fbeta after epoch 6: 0.6495575904846191: 
01/17/2020 15:17:53 - INFO - __main__ -   lr after epoch 6: 0.0
01/17/2020 15:17:53 - INFO - __main__ -   train_loss after epoch 6: 0.3744906851342905
01/17/2020 15:17:53 - INFO - __main__ -   



(2568, 0.4543692501047133)

# Save Model 

In [1]:
learner.save_model()

NameError: ignored

# Inference

In [0]:
from fast_bert.prediction import BertClassificationPredictor

In [126]:
MODEL_PATH = OUTPUT_PATH/'model_out'

predictor = BertClassificationPredictor(model_path=MODEL_PATH, 
                                        label_path=LABEL_PATH,
                                        multi_label=True,
                                        model_type='bert',
                                        do_lower_case=True)

01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -   Model name '/content/drive/My Drive/models/output/model_out' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1). Assuming '/content/drive/My Drive/models/output/model_out' is a path or url to a directory containing tokenizer files.
01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/models/output/model_out/vocab.txt
01/17/2020 13:51:40 - INFO - transformers.tokenization_utils -  

/content/drive/My Drive/models/output/model_out
<class 'str'>


In [142]:
single_pred = predictor.predict("Passed my exams! whoop whoop")

01/17/2020 13:58:50 - INFO - root -   Writing example 0 of 1


In [143]:
single_pred

[('joy', 0.7095755934715271),
 ('optimism', 0.5588766932487488),
 ('love', 0.24737906455993652),
 ('anticipation', 0.22631953656673431),
 ('sadness', 0.14842192828655243),
 ('disgust', 0.13481934368610382),
 ('anger', 0.1133059561252594),
 ('fear', 0.10953650623559952),
 ('trust', 0.10050033777952194),
 ('pessimism', 0.10005820542573929),
 ('surprise', 0.08825845271348953)]